In [5]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys
import os

ROOT = Path('../')
DATA_DIR = ROOT/'tests'/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'watchlist.yaml')
w


{'name': 'test',
 'email_addresses': ['brainbummer@mailinator.com', 'rhymedude@mailinator.com'],
 'products':     description       stock_code
 0   Brazil nuts           363477
 1  GB chocolate           260803
 2          fake  fake_stock_code
 3        cheese           281739}

In [8]:
# Get and parse a product

code = w['products']['stock_code'].iat[0]
response = cd.get_product(code)
cd.parse_product(response)

OrderedDict([('stock_code', '363477'),
             ('name', 'Ceres Organics Brazil Nuts 250g'),
             ('description',
              "Ceres organics brazil nuts are sourced from the pristine forests of the amazon, where they're sustainably grown and harvested. They're carefully selected, shelled and dried with minimal processing, to retain their natural goodness. Packed with the antioxidant selenium and a good source of the essential mineral magnesium, they make a nutritious snack. You can also enjoy their creamy, nutty deliciousness in salads, hot or cold cereals, or baked goods."),
             ('size', '250g'),
             ('sale_price', 13.4),
             ('price', 14.2),
             ('discount_percentage', 5.6),
             ('unit_price', '$5.36/100G'),
             ('datetime', '2020-06-11T17:14:53')])

In [10]:
# Get and parse many products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes)
f

CPU times: user 265 ms, sys: 11.9 ms, total: 277 ms
Wall time: 1.17 s


,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,363477,Ceres Organics Brazil Nuts 250g,Ceres organics brazil nuts are sourced from th...,250g,13.4,14.2,5.6,$5.36/100G,2020-06-11 17:15:03
1,260803,None,None,None,NaN,NaN,NaN,None,2020-06-11 17:15:04
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2020-06-11 17:15:04
3,281739,Mainland Cheese Block Organic Cheddar 500g,Mainland organic cheddar is a mild cheddar che...,500g,NaN,16.0,NaN,$32.00/1KG,2020-06-11 17:15:04


In [11]:
# Filter sales

cd.filter_sales(f)

,name,sale_price,price,discount_percentage
0,Ceres Organics Brazil Nuts 250g,13.4,14.2,5.6


In [12]:
# Email sales

domain = os.environ['MAILGUN_DOMAIN'] # replace with your Mailgun domain
key = os.environ['MAILGUN_KEY'] # replace with your Mailgun API key
cd.email(f, ['brainbummer@mailinator.com'], domain, key)

KeyError: 'MAILGUN_DOMAIN'

In [13]:
# Get products in bulk

cd.run_pipeline(DATA_DIR/'watchlist.yaml')

,stock_code,name,description,size,sale_price,price,discount_percentage,unit_price,datetime
0,363477,Ceres Organics Brazil Nuts 250g,Ceres organics brazil nuts are sourced from th...,250g,13.4,14.2,5.6,$5.36/100G,2020-06-11 17:15:25
1,260803,None,None,None,NaN,NaN,NaN,None,2020-06-11 17:15:26
2,fake_stock_code,None,None,None,NaN,NaN,NaN,None,2020-06-11 17:15:26
3,281739,Mainland Cheese Block Organic Cheddar 500g,Mainland organic cheddar is a mild cheddar che...,500g,NaN,16.0,NaN,$32.00/1KG,2020-06-11 17:15:26


In [ ]:
# Run pipeline again: read watchlist, collect products, write products to file, 
# filter sales, and email results

cd.run_pipeline(DATA_DIR/'watchlist.yaml', OUT_DIR/'products.csv', domain, key)
%less {OUT_DIR/'products.csv'}